# Lion Cheetah Classification task

Baed on the following Kaggle data set: https://www.kaggle.com/datasets/mikoajfish99/lions-or-cheetahs-image-classification

In [1]:
import tensorflow

: 

In [1]:
import tensorflow

: 

In [6]:
# In this case, we have two data sets separting cheetah and lions images

cheetah_folder = "./images/Cheetah"
lion_folder = "./images/Lions"

def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
                labels.append(folder.split('/')[-1])  # Assuming folder name is the label
    return images, labels

cheetah_images, cheetah_labels = load_images_from_folder(cheetah_folder)
lion_images, lion_labels = load_images_from_folder(lion_folder)

NameError: name 'os' is not defined